In [9]:
from pyproj import Proj, transform
from bokeh.models import (GeoJSONDataSource, Range1d, Circle, ColumnDataSource, GMapOptions, GMapPlot, 
                          DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, WMTSTileSource)
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.tile_providers import STAMEN_TONER, CARTODBPOSITRON, STAMEN_TERRAIN
import pandas as pd

output_notebook()

Loading BokehJS ...

In [10]:
data_urls = {
    'Water Sprays': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/5409b7735d384798b2a360aa47c9b128_0.csv',
    'Parking': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/962da9bb739f440ba33e746661921244_9.csv',
    'Fire Boxes': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/3a0f4db1e63a4a98a456fdb71dc37a81_4.csv',
    'Fire Hydrants': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/1b0717d5b4654882ae36adc4a20fd64b_0.csv',
    'Electric Vehicle Chargers': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/465e00f9632145a1ad645a27d27069b4_2.csv',
    'Fire Depts': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/092857c15cbb49e8b214ca5e228317a1_2.csv',
    'Police Stations': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/e5a0066d38ac4e2abbc7918197a4f6af_6.csv',
    'WickedWiFi (FREE!!)': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/4b803745fedd4e88861967d16a1e07fb_0.csv',
    'Trees': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/ce863d38db284efe83555caf8a832e2a_1.csv',
    'Snow Emergency Parking': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/53ebc23fcc654111b642f70e61c63852_0.csv',
    'Hubway Stations': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/ee7474e2a0aa45cbbdfe0b747a5eb032_0.csv',
    'Community Center Pools': 'http://bostonopendata-boston.opendata.arcgis.com/datasets/5575f763dbb64effa36acd67085ef3a8_0.csv',
}

In [11]:
from ipywidgets import interact, interactive, fixed, Dropdown

In [21]:
colors = ['blue', 'green', 'red', 'yellow']

@interact(data_src=sorted(data_urls), color=colors)
def plot_data(data_src, color):
    # Load data
    sprays = pd.read_csv(data_urls[data_src])

    # Convert EPSG code
    p1 = Proj(init='epsg:4326')  # this is the EPSG code of the original
    p2 = Proj(init='epsg:3857')  # this is the EPSG code of the tiles
    transformed_coords = [transform(p1, p2, x1, y1) for x1, y1 in zip(sprays['X'], sprays['Y'])]
    sprays['X'], sprays['Y'] = (zip(*transformed_coords))

    # Convert to coordinates
    src = ColumnDataSource(sprays)

    # Set x-range and y-range of map
    x_range = Range1d(start=sprays['X'].min(), end=sprays['X'].max(), bounds=None)
    y_range = Range1d(start=sprays['Y'].min(), end=sprays['Y'].max(), bounds=None)

    # Plot figure
    p = figure(x_range=x_range, y_range=y_range, tools='pan,wheel_zoom', 
               active_scroll="wheel_zoom", plot_height=400, plot_width=400)
    p.axis.visible = False
    circs = Circle(x='X', y='Y', size=10, line_color=None, fill_alpha=0.8, fill_color=color)
    p.add_glyph(src, circs)
    p.add_tile(STAMEN_TONER)

    show(p)